<a href="https://colab.research.google.com/github/panh1992/ML-000/blob/main/Week02/chap02_school_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chap02 作业

使用 cython 优化 target_encoding 代码，体验速度差距

<span id = "1"></span>
## 1、import 必要的模块，准备测试数据

In [1]:
import numpy as np
import pandas as pd

y = np.random.randint(2, size=(200, 1))
x = np.random.randint(10, size=(200, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

# 展示测试数据
data.head()

,y,x
0,1,9
1,0,0
2,0,4
3,0,9
4,1,3


## 2、 比较两个版本 python 代码差距  

### 版本一

In [2]:
def target_mean_v1(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  for i in range(data.shape[0]):
    groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
    result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
  return result

In [3]:
v1_result = target_mean_v1(data, 'y', 'x')

v1_result

array([0.55      , 0.45833333, 0.53333333, 0.6       , 0.6       ,
       0.5       , 0.54166667, 0.65      , 0.55      , 0.51724138,
       0.58823529, 0.6       , 0.54166667, 0.64705882, 0.65      ,
       0.55      , 0.35714286, 0.58823529, 0.54166667, 0.35714286,
       0.51724138, 0.6       , 0.45454545, 0.55      , 0.53333333,
       0.28571429, 0.35714286, 0.4375    , 0.54166667, 0.55      ,
       0.6       , 0.54166667, 0.45833333, 0.45454545, 0.5       ,
       0.35714286, 0.5       , 0.54545455, 0.53333333, 0.45833333,
       0.6       , 0.55172414, 0.45833333, 0.45833333, 0.4375    ,
       0.46666667, 0.6       , 0.54545455, 0.51724138, 0.45833333,
       0.45833333, 0.55172414, 0.55      , 0.45454545, 0.6       ,
       0.65      , 0.53333333, 0.45454545, 0.6       , 0.65      ,
       0.64705882, 0.35714286, 0.55172414, 0.6       , 0.5       ,
       0.53333333, 0.41666667, 0.65      , 0.45454545, 0.46666667,
       0.5       , 0.58823529, 0.54166667, 0.51724138, 0.55   

In [4]:
%%timeit
target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 916 ms per loop


### 版本二

In [5]:
def target_mean_v2(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i in range(data.shape[0]):
    if data.loc[i, x_name] not in value_dict.keys():
      value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] = 1
    else:
      value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] += 1
  for i in range(data.shape[0]):
    result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
  return result

In [6]:
v2_result = target_mean_v1(data, 'y', 'x')

v2_result

array([0.55      , 0.45833333, 0.53333333, 0.6       , 0.6       ,
       0.5       , 0.54166667, 0.65      , 0.55      , 0.51724138,
       0.58823529, 0.6       , 0.54166667, 0.64705882, 0.65      ,
       0.55      , 0.35714286, 0.58823529, 0.54166667, 0.35714286,
       0.51724138, 0.6       , 0.45454545, 0.55      , 0.53333333,
       0.28571429, 0.35714286, 0.4375    , 0.54166667, 0.55      ,
       0.6       , 0.54166667, 0.45833333, 0.45454545, 0.5       ,
       0.35714286, 0.5       , 0.54545455, 0.53333333, 0.45833333,
       0.6       , 0.55172414, 0.45833333, 0.45833333, 0.4375    ,
       0.46666667, 0.6       , 0.54545455, 0.51724138, 0.45833333,
       0.45833333, 0.55172414, 0.55      , 0.45454545, 0.6       ,
       0.65      , 0.53333333, 0.45454545, 0.6       , 0.65      ,
       0.64705882, 0.35714286, 0.55172414, 0.6       , 0.5       ,
       0.53333333, 0.41666667, 0.65      , 0.45454545, 0.46666667,
       0.5       , 0.58823529, 0.54166667, 0.51724138, 0.55   

In [7]:
%%timeit
target_mean_v2(data, 'y', 'x')

100 loops, best of 3: 10.9 ms per loop


## 3、使用 cython 优化 版本二 代码

In [8]:
%load_ext Cython

#### 1）单纯提取多余计算变量 比较速度差异

In [9]:
%%cython -a --cplus

import numpy as np
import pandas as pd

cpdef target_mean_v3(data, y_name, x_name):
  length = data.shape[0]
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()

  for i in range(length):
    x = data.loc[i, x_name]
    if x not in value_dict.keys():
      value_dict[x] = data.loc[i, y_name]
      count_dict[x] = 1
    else:
      value_dict[x] += data.loc[i, y_name]
      count_dict[x] += 1
  
  for i in range(length):
    x = data.loc[i, x_name]
    result[i] = (value_dict[x] - data.loc[i, y_name]) / (count_dict[x] - 1)
  return result

In [10]:
v3_result = target_mean_v3(data, 'y', 'x')

v3_result

array([0.55      , 0.45833333, 0.53333333, 0.6       , 0.6       ,
       0.5       , 0.54166667, 0.65      , 0.55      , 0.51724138,
       0.58823529, 0.6       , 0.54166667, 0.64705882, 0.65      ,
       0.55      , 0.35714286, 0.58823529, 0.54166667, 0.35714286,
       0.51724138, 0.6       , 0.45454545, 0.55      , 0.53333333,
       0.28571429, 0.35714286, 0.4375    , 0.54166667, 0.55      ,
       0.6       , 0.54166667, 0.45833333, 0.45454545, 0.5       ,
       0.35714286, 0.5       , 0.54545455, 0.53333333, 0.45833333,
       0.6       , 0.55172414, 0.45833333, 0.45833333, 0.4375    ,
       0.46666667, 0.6       , 0.54545455, 0.51724138, 0.45833333,
       0.45833333, 0.55172414, 0.55      , 0.45454545, 0.6       ,
       0.65      , 0.53333333, 0.45454545, 0.6       , 0.65      ,
       0.64705882, 0.35714286, 0.55172414, 0.6       , 0.5       ,
       0.53333333, 0.41666667, 0.65      , 0.45454545, 0.46666667,
       0.5       , 0.58823529, 0.54166667, 0.51724138, 0.55   

In [11]:
%%timeit
target_mean_v3(data, 'y', 'x')

100 loops, best of 3: 6.18 ms per loop


### 2）使用 numpy array 代替 pandas loc 查找 比较速度差异

In [12]:
%%cython -a --cplus

import numpy as np
import pandas as pd

cpdef target_mean_v4(data, y_name, x_name):

  length = data.shape[0]
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()

  x_index = data.columns.get_loc(x_name)
  y_index = data.columns.get_loc(y_name)
  matrix = data.values

  for i in range(length):
    x = matrix[i][x_index]
    if x not in value_dict.keys():
      value_dict[x] = matrix[i][y_index]
      count_dict[x] = 1
    else:
      value_dict[x] += matrix[i][y_index]
      count_dict[x] += 1
  
  for i in range(length):
    x = matrix[i][x_index]
    result[i] = (value_dict[x] - matrix[i][y_index]) / (count_dict[x] - 1)
  return result


In [13]:
v4_result = target_mean_v4(data, 'y', 'x')

v4_result

array([0.55      , 0.45833333, 0.53333333, 0.6       , 0.6       ,
       0.5       , 0.54166667, 0.65      , 0.55      , 0.51724138,
       0.58823529, 0.6       , 0.54166667, 0.64705882, 0.65      ,
       0.55      , 0.35714286, 0.58823529, 0.54166667, 0.35714286,
       0.51724138, 0.6       , 0.45454545, 0.55      , 0.53333333,
       0.28571429, 0.35714286, 0.4375    , 0.54166667, 0.55      ,
       0.6       , 0.54166667, 0.45833333, 0.45454545, 0.5       ,
       0.35714286, 0.5       , 0.54545455, 0.53333333, 0.45833333,
       0.6       , 0.55172414, 0.45833333, 0.45833333, 0.4375    ,
       0.46666667, 0.6       , 0.54545455, 0.51724138, 0.45833333,
       0.45833333, 0.55172414, 0.55      , 0.45454545, 0.6       ,
       0.65      , 0.53333333, 0.45454545, 0.6       , 0.65      ,
       0.64705882, 0.35714286, 0.55172414, 0.6       , 0.5       ,
       0.53333333, 0.41666667, 0.65      , 0.45454545, 0.46666667,
       0.5       , 0.58823529, 0.54166667, 0.51724138, 0.55   

In [14]:
%%timeit
target_mean_v4(data, 'y', 'x')

1000 loops, best of 3: 470 µs per loop


### 3）使用 numpy cimport 比较速度差异

In [15]:
%%cython -a --cplus

import numpy as np
cimport numpy as cnp
import pandas as pd

cpdef target_mean_v5(data, y_name, x_name):
  cdef long long length = data.shape[0]
  cdef cnp.ndarray result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()

  x_index = data.columns.get_loc(x_name)
  y_index = data.columns.get_loc(y_name)
  matrix = data.values

  for i in range(length):
    x = matrix[i][x_index]
    if x not in value_dict.keys():
      value_dict[x] = matrix[i][y_index]
      count_dict[x] = 1
    else:
      value_dict[x] += matrix[i][y_index]
      count_dict[x] += 1
  
  for i in range(length):
    x = matrix[i][x_index]
    result[i] = (value_dict[x] - matrix[i][y_index]) / (count_dict[x] - 1)
  return result

In [16]:
v5_result = target_mean_v5(data, 'y', 'x')

v5_result

array([0.55      , 0.45833333, 0.53333333, 0.6       , 0.6       ,
       0.5       , 0.54166667, 0.65      , 0.55      , 0.51724138,
       0.58823529, 0.6       , 0.54166667, 0.64705882, 0.65      ,
       0.55      , 0.35714286, 0.58823529, 0.54166667, 0.35714286,
       0.51724138, 0.6       , 0.45454545, 0.55      , 0.53333333,
       0.28571429, 0.35714286, 0.4375    , 0.54166667, 0.55      ,
       0.6       , 0.54166667, 0.45833333, 0.45454545, 0.5       ,
       0.35714286, 0.5       , 0.54545455, 0.53333333, 0.45833333,
       0.6       , 0.55172414, 0.45833333, 0.45833333, 0.4375    ,
       0.46666667, 0.6       , 0.54545455, 0.51724138, 0.45833333,
       0.45833333, 0.55172414, 0.55      , 0.45454545, 0.6       ,
       0.65      , 0.53333333, 0.45454545, 0.6       , 0.65      ,
       0.64705882, 0.35714286, 0.55172414, 0.6       , 0.5       ,
       0.53333333, 0.41666667, 0.65      , 0.45454545, 0.46666667,
       0.5       , 0.58823529, 0.54166667, 0.51724138, 0.55   

In [17]:
%%timeit
target_mean_v5(data, 'y', 'x')

1000 loops, best of 3: 449 µs per loop


## 总结

本次优化过程主要分为几步

1. python 代码逻辑简化
2. 使用 cython 提取多余计算变量
3. pandas loc 查找，替换为 numpy array 操作 替换后效率提升极大
4. 使用 cdef 定义精确类型 进一步提速